## Imports

In [20]:
import copy
import glob
import importlib
import math
import os
import random
import shutil
import sys
from math import degrees, pi, radians
from os.path import *

import keras
import mahotas.features as mah
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import SimpleITK as sitk
import skimage.feature as skf
from skimage.morphology import ball

import cnn_builder as cbuild
import config
import lipiodol_methods as lm
import niftiutils.helper_fxns as hf
import niftiutils.liver_metrics as lmet
import niftiutils.masks as masks
import niftiutils.registration as reg
import niftiutils.transforms as tr
from config import Config

%matplotlib inline

In [3]:
importlib.reload(hf)
C = config.Config()

In [4]:
img_dir = "D:\\Lipiodol\\Images all"
seg_dir = "D:\\Lipiodol\\Images extracted and segmentations"
target_dir = "D:\\Lipiodol\\Data"

## Pattern analysis

In [ ]:
(PK2 image)

In [ ]:
patient_id = "BM-05"
importlib.reload(lm)
paths = lm.get_paths(patient_id, target_dir, check_valid=False)

mask_dir, nii_dir, ct24_path, ct24_tumor_mask_path, ct24_liver_mask_path, \
mribl_art_path, mribl_pre_path, \
mribl_tumor_mask_path, mribl_liver_mask_path, \
mribl_enh_mask_path, mribl_nec_mask_path, \
mri30d_art_path, mri30d_pre_path, \
mri30d_tumor_mask_path, mri30d_liver_mask_path, \
mri30d_enh_mask_path, mri30d_nec_mask_path, \
ball_ct24_path, ball_mribl_path, ball_mri30d_path, \
ball_mask_path, ball_mribl_enh_mask_path, ball_mri30d_enh_mask_path, \
midlip_mask_path, ball_midlip_mask_path, \
highlip_mask_path, ball_highlip_mask_path = paths

### Build DataFrame

In [5]:
df = pd.DataFrame(columns=["PatientId", "enhancing_vol%", "lipcoverage_vol%",
                           "rim_enhancing%", "rim_lipiodol%",
                           "mribl_sum_entropy", "mribl_entropy", "mribl_diff_entropy",
                           "midlip_sum_entropy", "midlip_entropy", "midlip_diff_entropy"])

In [22]:
for fn in glob.glob(join(target_dir,"*")):
    patient_id = basename(fn)
    print(patient_id)
    paths = lm.get_paths(patient_id, target_dir)

    mask_dir, nii_dir, ct24_path, ct24_tumor_mask_path, ct24_liver_mask_path, \
    mribl_art_path, mribl_pre_path, \
    mribl_tumor_mask_path, mribl_liver_mask_path, \
    mribl_enh_mask_path, mribl_nec_mask_path, \
    mri30d_art_path, mri30d_pre_path, \
    mri30d_tumor_mask_path, mri30d_liver_mask_path, \
    mri30d_enh_mask_path, mri30d_nec_mask_path, \
    ball_ct24_path, ball_mribl_path, ball_mri30d_path, \
    ball_mask_path, ball_mribl_enh_mask_path, ball_mri30d_enh_mask_path, \
    midlip_mask_path, ball_midlip_mask_path, \
    highlip_mask_path, ball_highlip_mask_path = paths
    
    df.loc[patient_id] = get_row()

BM-02


IndexError: boolean index did not match indexed array along dimension 2; dimension is 1 but corresponding boolean dimension is 59

In [ ]:
importlib.reload(lm)
lm.spherize(patient_id, target_dir)

In [12]:
ball, _ = masks.get_mask(ball_mask_path)

In [15]:
img, dims = hf.nii_load(ball_ct24_path)

In [16]:
img.shape

(59, 59, 1)

In [10]:
def get_row():
    row = []
    row = get_vol_coverage(row)
    
    ball_IV = lm.get_avg_ball_intensity(ball_ct24_path, ball_mask_path)
    #core_IV = lm.get_avg_core_intensity(ball_ct24_path, ball_mask_path)
    row = get_rim_coverage(row, hf.nii_load(ball_ct24_path)[0], core_IV)
    row = get_rim_coverage(row, masks.get_mask(ball_mribl_enh_mask_path)[0] + 1, 1.5)
    row = get_texture_feats(row, ball_mribl_enh_mask_path)
    row = get_texture_feats(row, ball_midlip_mask_path)

    return row

def get_vol_coverage(row):
    ball_mask,_ = masks.get_mask(ball_mask_path)
    ball_mask = ball_mask/ball_mask.max()

    mask,_ = masks.get_mask(ball_mribl_enh_mask_path)
    row.append(mask.sum()/ball_mask.sum())

    mask,_ = masks.get_mask(ball_midlip_mask_path)
    row.append(mask.sum()/ball_mask.sum())
    return row

def get_rim_coverage(row, img, threshold):
    IVs = lm.calc_intensity_shells_angles(img, ball_mask_path)
    IVs[IVs==0] = np.nan

    samples = lm.fibonacci_sphere(2500, True, randomize=True)
    samples = np.round(samples).astype(int)
    s0 = samples[:,0]
    s1 = samples[:,1]
    #for i in range(IVs.shape[-1]):
    #    print(np.nanmean(IVs[s0,s1,i]))

    rim_percent = 0
    for i in range(5):
        num,den=0,0
        for j in range(len(s0)):
            if not np.isnan(IVs[s0[j],s1[j],i]):
                den += 1
                if IVs[s0[j],s1[j],i] > threshold:
                    num += 1
        rim_percent = max([rim_percent, num/den])
    row.append(rim_percent)

    return row

def get_texture_feats(row, img):
    mask,_ = masks.get_mask(img)

    feats = mah.haralick(mask)
    sum_entropy = feats[:,7].mean()
    entropy = feats[:,8].mean()
    diff_entropy = feats[:,10].mean()

    row += [sum_entropy, entropy, diff_entropy]
    
    return row

## Figures

In [ ]:
lm.draw_unreg_fig(mribl_art_path, mribl_enh_mask_path, "D:\\Lipiodol\\Figures\\unreg\\MRIBL", 'b', 'mr')
lm.draw_unreg_fig(mri30d_art_path, mri30d_enh_mask_path, "D:\\Lipiodol\\Figures\\unreg\\MRI30d", 'r', 'mr')
lm.draw_unreg_fig(ct24_path, highlip_mask_path, "D:\\Lipiodol\\Figures\\unreg\\Lip", 'g', 'ct')

In [ ]:
lm.draw_reg_fig(ball_mribl_path, ball_mribl_enh_mask_path, "D:\\Lipiodol\\Figures\\MRIBL", 'b', 'mr')
lm.draw_reg_fig(ball_mri30d_path, ball_mri30d_enh_mask_path, "D:\\Lipiodol\\Figures\\MRI30d", 'r', 'mr')
lm.draw_reg_fig(ball_ct24_path, ball_highlip_mask_path, "D:\\Lipiodol\\Figures\\Lip", 'g', 'ct')

In [ ]:
# Figure for 24h CT
img,_ = hf.nii_load(ball_ct24_path)
blmask,_ = masks.get_mask(ball_mribl_enh_mask_path)
fumask,_ = masks.get_mask(ball_mri30d_enh_mask_path)

for sl in range(img.shape[-1]//4,img.shape[-1]*3//4,img.shape[-1]//12):
    plt.close()
    plt.imshow(img[...,sl], cmap='gray', vmin=30, vmax=250)
    FU=plt.contour(fumask[:,:,sl], colors='r', alpha=.4)
    plt.contour(blmask[:,:,sl], colors='b', alpha=.4)
    plt.contourf(fumask[:,:,sl], colors=[(0,0,0,0)]*6+[(1,0,0,.2)]) #look at the length of FU.allsegs
    plt.contourf(blmask[:,:,sl], colors=[(0,0,0,0)]*6+[(0,0,1,.2)])
    plt.axis('off')
    plt.savefig("D:\\Lipiodol\\Figures\\24hCT_%d.png" % sl, dpi=100, bbox_inches='tight')

## Create mini-DICOMs

In [ ]:
output_directory = "D:\\test"

In [ ]:
import SimpleITK as sitk
import sys, time, os
data_directory = "D:\\test"
series_reader = sitk.ImageSeriesReader()
series_IDs = series_reader.GetGDCMSeriesIDs(data_directory)
if not series_IDs:
    print("ERROR: given directory \""+data_directory+"\" does not contain a DICOM series.")
    sys.exit(1)
series_file_names = series_reader.GetGDCMSeriesFileNames(data_directory, series_IDs[0])

image_reader = sitk.ImageFileReader()
image_reader.LoadPrivateTagsOn()
image_list = []
for file_name in series_file_names:
    image_reader.SetFileName(file_name)
    image_list.append(image_reader.Execute())

# Pasting all of the slices into a 3D volume requires 2D image slices and not 3D slices
# The volume's origin and direction are taken from the first slice and the spacing from
# the difference between the first two slices. Note that we are assuming we are
# dealing with a volume represented by axial slices (z spacing is difference between images).
image_list2D = [image[:,:,0] for image in image_list]
image3D = sitk.JoinSeries(image_list2D, image_list[0].GetOrigin()[2], image_list[1].GetOrigin()[2] - image_list[0].GetOrigin()[2])
image3D.SetDirection(image_list[0].GetDirection())

# Modify the image (blurring)
filtered_image = sitk.DiscreteGaussian(image3D)

# Write the 3D image as a series
# IMPORTANT: There are many DICOM tags that need to be updated when you modify an
#            original image. This is a delicate opration and requires knowlege of
#            the DICOM standard. This example only modifies some. For a more complete
#            list of tags that need to be modified see:
#                           http://gdcm.sourceforge.net/wiki/index.php/Writing_DICOM

writer = sitk.ImageFileWriter()
# Use the study/seriers/frame of reference information given in the meta-data
# dictionary and not the automatically generated information from the file IO
writer.KeepOriginalImageUIDOn()
modification_time = time.strftime("%H%M%S")
modification_date = time.strftime("%Y%m%d")
for i in range(filtered_image.GetDepth()):
    image_slice = filtered_image[:,:,i]
    original_slice = image_list[i]
    # Copy the meta-data except the rescale-intercept, rescale-slope
    for k in original_slice.GetMetaDataKeys():
        if k!="0028|1052" and k!= "0028|1053":
            image_slice.SetMetaData(k, original_slice.GetMetaData(k))
    # Set relevant keys indicating the change, modify or remove private tags as needed
    image_slice.SetMetaData("0008|0031", modification_time)
    image_slice.SetMetaData("0008|0021", modification_date)
    image_slice.SetMetaData("0008|0008", "DERIVED\SECONDARY")
    # Each of the UID components is a number (cannot start with zero) and separated by a '.'
    # We create a unique series ID using the date and time.
    image_slice.SetMetaData("0020|000e", "1.2.826.0.1.3680043.2.1125."+modification_date+".1"+modification_time)
    # Write to the output directory and add the extension dcm if not there, to force writing is in DICOM format.
    writer.SetFileName(os.path.join(sys.argv[2], os.path.basename(series_file_names[i])) + ('' if os.path.splitext(series_file_names[i])[1] == '.dcm' else '.dcm'))
    writer.Execute(image_slice)

## Extra

IVs = get_intensity_section(ball_ct24_path, ball_mask_path, params)
            if highest_I_V < I_V:
                best_params = params
                highest_I_V = I_V
                print(best_params)
theta1_best, phi1_best, dtheta_best, dz_best = best_params

In [ ]:
ball_IV = get_avg_ball_intensity(ball_ct24_path, ball_mask_path)
core_IV = get_avg_core_intensity(ball_ct24_path, ball_mask_path)

In [303]:
ball_IV, core_IV

(197.83461276161094, 209.89937589866474)

In [ ]:
importlib.reload(lm)
IVs = lm.calc_intensity_shells_angles(hf.nii_load(ball_ct24_path)[0], ball_mask_path)
IVs[IVs==0] = np.nan

samples = lm.fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(IVs.shape[-1]):
    print(np.nanmean(IVs[s0,s1,i]))

In [388]:
loopIVs = np.tile(IVs[:-1,:-1,:], (2,2,1))
loopIVs[180:,:,:] = np.nan

In [ ]:
for dtheta in range(30,180,20):
    print(dtheta)
    best_IV = core_IV
    worst_IV = core_IV
    for theta in range(0,180,20):
        for phi in range(180,480,20):
            for shell1 in range(4):
                for shell2 in range(shell1+1,5):
                    IV = np.nanmean(loopIVs[max(theta-dtheta,0) : theta+dtheta, phi-dtheta : phi+dtheta, shell1:shell2])
                    params = theta, phi % 360, shell1, shell2
                    if best_IV < IV:
                        best_params = params
                        best_IV = IV
                        
            IV = np.nanmean(loopIVs[max(theta-dtheta,0) : theta+dtheta, phi-dtheta : phi+dtheta, :5])
            params = theta, phi % 360
            if worst_IV > IV:
                worst_params = params
                worst_IV = IV
                
    print(best_params, round(best_IV,1))
    print(worst_params, round(worst_IV,1))

In [ ]:
importlib.reload(lm)
enhmask_density = lm.calc_intensity_shells_angles(masks.get_mask(ball_mribl_enh_mask_path)[0]+1, ball_mask_path)

enhmask_density = enhmask_density[...,:5]
enhmask_density[enhmask_density==0] = np.nan

samples = fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
importlib.reload(lm)
enhmask_density = lm.calc_intensity_shells_angles(masks.get_mask(ball_mri30d_enh_mask_path)[0]+1, ball_mask_path)

enhmask_density = enhmask_density[...,:5]
enhmask_density[enhmask_density==0] = np.nan

samples = fibonacci_sphere(2500, True, randomize=True)
samples = np.round(samples).astype(int)
s0 = samples[:,0]
s1 = samples[:,1]
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
for i in range(enhmask_density.shape[-1]):
    print(np.nanmean(enhmask_density[s0,s1,i]))

In [ ]:
enhmask_density =

In [ ]:
conv_rate = np.zeros((181,360))